# Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from scipy.stats import norm

# Setup

In [2]:
def maxnorm(elo1, elo2, var=1500): # 2000 looks best
    #elo1+=var/10
    nm=elo1-elo2
    var*=2
    return 1-norm.cdf(-nm/var**.5)

In [3]:
NBA_data= pd.read_csv("nbaallelo.csv")
NBA_data

gameorder       game_id lg_id  _iscopy  year_id  date_game  \
0               1  194611010TRH   NBA        0     1947  11/1/1946   
1               1  194611010TRH   NBA        1     1947  11/1/1946   
2               2  194611020CHS   NBA        0     1947  11/2/1946   
3               2  194611020CHS   NBA        1     1947  11/2/1946   
4               3  194611020DTF   NBA        0     1947  11/2/1946   
...           ...           ...   ...      ...      ...        ...   
126309      63155  201506110CLE   NBA        0     2015  6/11/2015   
126310      63156  201506140GSW   NBA        0     2015  6/14/2015   
126311      63156  201506140GSW   NBA        1     2015  6/14/2015   
126312      63157  201506170CLE   NBA        0     2015  6/16/2015   
126313      63157  201506170CLE   NBA        1     2015  6/16/2015   

        seasongame  is_playoffs team_id    fran_id  ...  win_equiv  opp_id  \
0                1            0     TRH    Huskies  ...  40.294830     NYK   
1                1            0     NYK     Knicks  ...  41.705170     TRH   
2                1            0     CHS      Stags  ...  42.012257     NYK   
3                2            0     NYK     Knicks  ...  40.692783     CHS   
4                1            0     DTF    Falcons  ...  38.864048     WSC   
...            ...          ...     ...        ...  ...        ...     ...   
126309         100            1     CLE  Cavaliers  ...  60.309792     GSW   
126310         102            1     GSW   Warriors  ...  68.013329     CLE   
126311         101            1     CLE  Cavaliers  ...  60.010067     GSW   
126312         102            1     CLE  Cavaliers  ...  59.290245     GSW   
126313         103            1     GSW   Warriors  ...  68.519516     CLE   

         opp_fran  opp_pts  opp_elo_i  opp_elo_n  game_location  game_result  \
0          Knicks       68  1300.0000  1306.7233              H            L   
1         Huskies       66  1300.0000  1293.2767              A            W   
2          Knicks       47  1306.7233  1297.0712              H            W   
3           Stags       63  1300.0000  1309.6521              A            L   
4        Capitols       50  1300.0000  1320.3811              H            L   
...           ...      ...        ...        ...            ...          ...   
126309   Warriors      103  1790.9591  1809.9791              H            L   
126310  Cavaliers       91  1704.3949  1700.7391              H            W   
126311   Warriors      104  1809.9791  1813.6349              A            L   
126312   Warriors      105  1813.6349  1822.2881              H            L   
126313  Cavaliers       97  1700.7391  1692.0859              A            W   

        forecast notes  
0       0.640065   NaN  
1       0.359935   NaN  
2       0.631101   NaN  
3       0.368899   NaN  
4       0.640065   NaN  
...          ...   ...  
126309  0.546572   NaN  
126310  0.765565   NaN  
126311  0.234435   NaN  
126312  0.481450   NaN  
126313  0.518550   NaN  

[126314 rows x 23 columns]

In [4]:
NBA_data.columns

Index(['gameorder', 'game_id', 'lg_id', '_iscopy', 'year_id', 'date_game',
       'seasongame', 'is_playoffs', 'team_id', 'fran_id', 'pts', 'elo_i',
       'elo_n', 'win_equiv', 'opp_id', 'opp_fran', 'opp_pts', 'opp_elo_i',
       'opp_elo_n', 'game_location', 'game_result', 'forecast', 'notes'],
      dtype='object')

In [5]:
NBA_data['maxnorm_forecast'] = NBA_data.apply(lambda x: maxnorm(x['elo_i'], x['opp_elo_i']), axis=1)
NBA_data['results'] = (NBA_data['game_result'] == 'W')*1

In [6]:
NBA_var=np.var(NBA_data['elo_i'])
NBA_mean=np.mean(NBA_data[NBA_data['team_id']=='NYK']['elo_i'])

print(NBA_mean, NBA_var) #/(126314**.5)

1497.6156040908302 12575.267726994409


In [7]:
NBA_data[['elo_i', 'opp_elo_i', 'forecast', 'maxnorm_forecast', 'results']]

elo_i  opp_elo_i  forecast  maxnorm_forecast  results
0       1300.0000  1300.0000  0.640065          0.500000        0
1       1300.0000  1300.0000  0.359935          0.500000        1
2       1300.0000  1306.7233  0.631101          0.451152        1
3       1306.7233  1300.0000  0.368899          0.548848        0
4       1300.0000  1300.0000  0.640065          0.500000        0
...           ...        ...       ...               ...      ...
126309  1723.4149  1790.9591  0.546572          0.108754        0
126310  1809.9791  1704.3949  0.765565          0.973053        1
126311  1704.3949  1809.9791  0.234435          0.026947        0
126312  1700.7391  1813.6349  0.481450          0.019643        0
126313  1813.6349  1700.7391  0.518550          0.980357        1

[126314 rows x 5 columns]

In [8]:
regdf1=NBA_data[['game_id','forecast', 'maxnorm_forecast']]
model1 = smf.ols(formula='maxnorm_forecast ~ forecast', data=regdf1).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       maxnorm_forecast   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.584
Method:                 Least Squares   F-statistic:                 1.772e+05
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:04:17   Log-Likelihood:                -12046.
No. Observations:              126314   AIC:                         2.410e+04
Df Residuals:                  126312   BIC:                         2.412e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2323      0.002   -122.658      0.000      -0.236      -0.229
forecast       1.4647      0.003    420.942      0.000       1.458       1.471
==============================================================================
Omnibus:                     7206.056   Durbin-Watson:                   2.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2697.663
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                       2.284   Cond. No.                         5.85
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 538 Model Test Results

In [9]:
regdf2=NBA_data[['game_id','forecast', 'results']]
model2 = smf.ols(formula='forecast ~ results', data=regdf2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               forecast   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                 2.864e+04
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:04:17   Log-Likelihood:                 27686.
No. Observations:              126314   AIC:                        -5.537e+04
Df Residuals:                  126312   BIC:                        -5.535e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4075      0.001    526.892      0.000       0.406       0.409
results        0.1851      0.001    169.227      0.000       0.183       0.187
==============================================================================
Omnibus:                     6278.891   Durbin-Watson:                   3.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2485.259
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                       2.313   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
log_reg1 = sm.Logit(regdf2['results'], regdf2['forecast']).fit() 
log_reg1.summary()

Optimization terminated successfully.
         Current function value: 0.678589
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                 0.02100
Time:                        18:04:18   Log-Likelihood:                -85715.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
forecast       0.6341      0.011     59.958      0.000       0.613       0.655
==============================================================================
"""

# Maxnorm Test Results

In [11]:
regdf3=NBA_data[['game_id','maxnorm_forecast', 'results']]
model3 = smf.ols(formula='maxnorm_forecast ~ results', data=regdf3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       maxnorm_forecast   R-squared:                       0.106
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                 1.494e+04
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:04:18   Log-Likelihood:                -60352.
No. Observations:              126314   AIC:                         1.207e+05
Df Residuals:                  126312   BIC:                         1.207e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3658      0.002    235.611      0.000       0.363       0.369
results        0.2684      0.002    122.231      0.000       0.264       0.273
==============================================================================
Omnibus:                  1683684.304   Durbin-Watson:                   2.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9813.642
Skew:                           0.000   Prob(JB):                         0.00
Kurtosis:                       1.634   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
log_reg2 = sm.Logit(regdf3['results'], regdf3['maxnorm_forecast']).fit() 
log_reg2.summary()

Optimization terminated successfully.
         Current function value: 0.671404
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                 0.03137
Time:                        18:04:18   Log-Likelihood:                -84808.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
maxnorm_forecast     0.6579      0.009     72.470      0.000       0.640       0.676
====================================================================================
"""

# Weighted Mix

In [13]:
a=.1
NBA_data['w_forecast']= NBA_data['maxnorm_forecast']*a+ NBA_data['forecast']*(1-a)

In [14]:
regdf4=NBA_data[['game_id','w_forecast', 'results','maxnorm_forecast','forecast']]
model4 = smf.ols(formula='w_forecast ~ results', data=regdf4).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             w_forecast   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                 2.806e+04
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:04:18   Log-Likelihood:                 20837.
No. Observations:              126314   AIC:                        -4.167e+04
Df Residuals:                  126312   BIC:                        -4.165e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4033      0.001    493.983      0.000       0.402       0.405
results        0.1934      0.001    167.511      0.000       0.191       0.196
==============================================================================
Omnibus:                     8140.394   Durbin-Watson:                   3.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2894.198
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                       2.258   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
log_reg3 = sm.Logit(regdf3['results'], regdf4[['forecast','maxnorm_forecast']]).fit() 
log_reg3.summary()

Optimization terminated successfully.
         Current function value: 0.670652
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126312
Method:                           MLE   Df Model:                            1
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                 0.03245
Time:                        18:04:18   Log-Likelihood:                -84713.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
forecast            -0.3309      0.024    -13.774      0.000      -0.378      -0.284
maxnorm_forecast     0.9096      0.020     44.513      0.000       0.870       0.950
====================================================================================
"""

In [16]:
log_reg4 = sm.Logit(regdf4['results'], regdf4['w_forecast']).fit() 
log_reg4.summary()

Optimization terminated successfully.
         Current function value: 0.677506
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                 0.02257
Time:                        18:04:18   Log-Likelihood:                -85578.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
w_forecast     0.6525      0.011     62.074      0.000       0.632       0.673
==============================================================================
"""

# Binned Testing 538

In [17]:
bindff=NBA_data[['forecast','results']]
bins= np.arange(0,1.01,.05)
labels= np.arange(0,100,5)
print(bins)
bindff['binned']=pd.cut(bindff['forecast'],bins=bins,labels=labels)
bindff['binned']=bindff['binned'].fillna(0)
bindff

[0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]


C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


forecast  results binned
0       0.640065        0     60
1       0.359935        1     35
2       0.631101        1     60
3       0.368899        0     35
4       0.640065        0     60
...          ...      ...    ...
126309  0.546572        0     50
126310  0.765565        1     75
126311  0.234435        0     20
126312  0.481450        0     45
126313  0.518550        1     50

[126314 rows x 3 columns]

In [18]:
bin_compf=[]
for i in range(len(bins)-1):
    binaver= np.mean(bindff['results'][bindff['binned']==labels[i]])
    bin_compf.append([(bins[i]+bins[i+1])/2,binaver])
bin_compf

[[0.025, 0.031055900621118012],
 [0.07500000000000001, 0.07574094401756312],
 [0.125, 0.1243680485338726],
 [0.17500000000000002, 0.17873542917696927],
 [0.225, 0.2273752466873414],
 [0.275, 0.27632074287865477],
 [0.32500000000000007, 0.3236698878151727],
 [0.375, 0.37402207694780837],
 [0.42500000000000004, 0.43101182654402104],
 [0.475, 0.48166611277279275],
 [0.525, 0.5183338872272073],
 [0.5750000000000001, 0.568988173455979],
 [0.625, 0.6259779230521916],
 [0.675, 0.6763301121848273],
 [0.7250000000000001, 0.7236792571213452],
 [0.775, 0.7726247533126586],
 [0.8250000000000001, 0.8212645708230307],
 [0.875, 0.8756319514661274],
 [0.925, 0.9242590559824369],
 [0.9750000000000001, 0.968944099378882]]

# Binned Testing

In [19]:
bindf=NBA_data[['maxnorm_forecast','results']]

In [20]:
bins= np.arange(0,1.01,.05)
labels= np.arange(0,100,5)
print(bins)
bindf['binned']=pd.cut(bindf['maxnorm_forecast'],bins=bins,labels=labels)

[0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]


C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [21]:
bindf['binned']=bindf['binned'].fillna(0)

C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
bindf

maxnorm_forecast  results binned
0               0.500000        0     45
1               0.500000        1     45
2               0.451152        1     45
3               0.548848        0     50
4               0.500000        0     45
...                  ...      ...    ...
126309          0.108754        0     10
126310          0.973053        1     95
126311          0.026947        0      0
126312          0.019643        0      0
126313          0.980357        1     95

[126314 rows x 3 columns]

In [23]:
#set(bindf['binned'])

In [24]:
bin_comp=[]
for i in range(len(bins)-1):
    binaver= np.mean(bindf['results'][bindf['binned']==labels[i]])
    bin_comp.append([(bins[i]+bins[i+1])/2,binaver])

In [25]:
bin_comp

[[0.025, 0.2819040401096018],
 [0.07500000000000001, 0.3945298534143725],
 [0.125, 0.4296974319857615],
 [0.17500000000000002, 0.42237640936686904],
 [0.225, 0.4528301886792453],
 [0.275, 0.45802856147683735],
 [0.32500000000000007, 0.45244512899499423],
 [0.375, 0.4945054945054945],
 [0.42500000000000004, 0.48677685950413224],
 [0.475, 0.49898744430943703],
 [0.525, 0.5010191602119853],
 [0.5750000000000001, 0.5132231404958678],
 [0.625, 0.5054945054945055],
 [0.675, 0.5475548710050058],
 [0.7250000000000001, 0.5419714385231627],
 [0.775, 0.5471698113207547],
 [0.8250000000000001, 0.577623590633131],
 [0.875, 0.5703025680142385],
 [0.925, 0.6054701465856275],
 [0.9750000000000001, 0.7180959598903982]]

# Weighted Binned Testing

In [26]:
bindfw=NBA_data[['w_forecast','results']]
bins= np.arange(0,1.01,.05)
labels= np.arange(0,100,5)
print(bins)
bindfw['binned']=pd.cut(bindfw['w_forecast'],bins=bins,labels=labels)
bindfw['binned']=bindfw['binned'].fillna(0)
bindfw

[0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]


C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


w_forecast  results binned
0         0.626059        0     60
1         0.373941        1     35
2         0.613106        1     60
3         0.386894        0     35
4         0.626059        0     60
...            ...      ...    ...
126309    0.502790        0     50
126310    0.786314        1     75
126311    0.213686        0     20
126312    0.435269        0     40
126313    0.564731        1     55

[126314 rows x 3 columns]

In [27]:
bin_compw=[]
for i in range(len(bins)-1):
    binaver= np.mean(bindfw['results'][bindfw['binned']==labels[i]])
    bin_compw.append([(bins[i]+bins[i+1])/2,binaver])
bin_compw

[[0.025, 0.03875968992248062],
 [0.07500000000000001, 0.08231954582319546],
 [0.125, 0.14187271990271585],
 [0.17500000000000002, 0.19902983977656916],
 [0.225, 0.24350693967294215],
 [0.275, 0.2977275819839434],
 [0.32500000000000007, 0.33852780945141264],
 [0.375, 0.38774264436576733],
 [0.42500000000000004, 0.4380792853154662],
 [0.475, 0.4766913963016277],
 [0.525, 0.5233086036983723],
 [0.5750000000000001, 0.5619207146845338],
 [0.625, 0.6122573556342327],
 [0.675, 0.6614721905485873],
 [0.7250000000000001, 0.7022724180160566],
 [0.775, 0.7564930603270579],
 [0.8250000000000001, 0.8009701602234308],
 [0.875, 0.8581272800972841],
 [0.925, 0.9176804541768045],
 [0.9750000000000001, 0.9612403100775194]]

# End